## Load Previous Day

This function queries the entsoe dataset and upload the previous day's energy demand to google cloud storage.

In [44]:
from google.cloud import storage
from google.oauth2 import service_account
import os
from entsoe import EntsoePandasClient
import pandas as pd
from datetime import datetime, timedelta

os.environ['GOOGLE_APPLICATION_CREDENTIALS']


'/Users/ns/github-repos/energy-dashboard/energy-dashboard.json'

In [32]:
client = EntsoePandasClient(api_key='909addb7-e4ae-4702-acc7-6b4f4fd9667b')
data = client.query_load("ES", 
                         start=pd.Timestamp("20200303T0000", tz='UTC'), 
                         end=pd.Timestamp("20200303T2300", tz='UTC'))

In [33]:
data

2020-03-03 01:00:00+01:00    24607.0
2020-03-03 02:00:00+01:00    23619.0
2020-03-03 03:00:00+01:00    23260.0
2020-03-03 04:00:00+01:00    23274.0
2020-03-03 05:00:00+01:00    23846.0
2020-03-03 06:00:00+01:00    26705.0
2020-03-03 07:00:00+01:00    30842.0
2020-03-03 08:00:00+01:00    33267.0
2020-03-03 09:00:00+01:00    34482.0
2020-03-03 10:00:00+01:00    34712.0
2020-03-03 11:00:00+01:00    34789.0
2020-03-03 12:00:00+01:00    34579.0
2020-03-03 13:00:00+01:00    34261.0
2020-03-03 14:00:00+01:00    33071.0
2020-03-03 15:00:00+01:00    32221.0
2020-03-03 16:00:00+01:00    31514.0
2020-03-03 17:00:00+01:00    31315.0
2020-03-03 18:00:00+01:00    31304.0
2020-03-03 19:00:00+01:00    33688.0
2020-03-03 20:00:00+01:00    34965.0
2020-03-03 21:00:00+01:00    33910.0
2020-03-03 22:00:00+01:00    31371.0
2020-03-03 23:00:00+01:00    28475.0
dtype: float64

In [36]:
creds = service_account.Credentials.from_service_account_file(os.environ['GOOGLE_APPLICATION_CREDENTIALS'])
project = 'ml-energy-dashboard'
client_s = storage.Client(credentials=creds, project=project)

In [40]:
def query_entsoe_data(date):
    client = EntsoePandasClient(api_key=os.environ['ENTSOE_TOKEN'])
    data = client.query_load("ES", 
                             start=pd.Timestamp(f"{date}T0000", tz='UTC'), 
                             end=pd.Timestamp(f"{date}T2300", tz='UTC'))
    return data

In [41]:
query_entsoe_data(20200303)

2020-03-03 01:00:00+01:00    24607.0
2020-03-03 02:00:00+01:00    23619.0
2020-03-03 03:00:00+01:00    23260.0
2020-03-03 04:00:00+01:00    23274.0
2020-03-03 05:00:00+01:00    23846.0
2020-03-03 06:00:00+01:00    26705.0
2020-03-03 07:00:00+01:00    30842.0
2020-03-03 08:00:00+01:00    33267.0
2020-03-03 09:00:00+01:00    34482.0
2020-03-03 10:00:00+01:00    34712.0
2020-03-03 11:00:00+01:00    34789.0
2020-03-03 12:00:00+01:00    34579.0
2020-03-03 13:00:00+01:00    34261.0
2020-03-03 14:00:00+01:00    33071.0
2020-03-03 15:00:00+01:00    32221.0
2020-03-03 16:00:00+01:00    31514.0
2020-03-03 17:00:00+01:00    31315.0
2020-03-03 18:00:00+01:00    31304.0
2020-03-03 19:00:00+01:00    33688.0
2020-03-03 20:00:00+01:00    34965.0
2020-03-03 21:00:00+01:00    33910.0
2020-03-03 22:00:00+01:00    31371.0
2020-03-03 23:00:00+01:00    28475.0
dtype: float64

In [49]:
def generate_date():
    return (datetime.today()+timedelta(-1)).strftime('%Y%m%d')

In [50]:
generate_date()

'20200303'

In [51]:
date = generate_date()
query_entsoe_data(date)

2020-03-03 01:00:00+01:00    24607.0
2020-03-03 02:00:00+01:00    23619.0
2020-03-03 03:00:00+01:00    23260.0
2020-03-03 04:00:00+01:00    23274.0
2020-03-03 05:00:00+01:00    23846.0
2020-03-03 06:00:00+01:00    26705.0
2020-03-03 07:00:00+01:00    30842.0
2020-03-03 08:00:00+01:00    33267.0
2020-03-03 09:00:00+01:00    34482.0
2020-03-03 10:00:00+01:00    34712.0
2020-03-03 11:00:00+01:00    34789.0
2020-03-03 12:00:00+01:00    34579.0
2020-03-03 13:00:00+01:00    34261.0
2020-03-03 14:00:00+01:00    33071.0
2020-03-03 15:00:00+01:00    32221.0
2020-03-03 16:00:00+01:00    31514.0
2020-03-03 17:00:00+01:00    31315.0
2020-03-03 18:00:00+01:00    31304.0
2020-03-03 19:00:00+01:00    33688.0
2020-03-03 20:00:00+01:00    34965.0
2020-03-03 21:00:00+01:00    33910.0
2020-03-03 22:00:00+01:00    31371.0
2020-03-03 23:00:00+01:00    28475.0
dtype: float64

In [52]:
def gcs_file_name(date):
    return f'es-energy-demand-raw-{date}'

In [54]:
bucket_name = 'ml-energy-dashboard-raw-data'
bucket = client_s.get_bucket(bucket_name)
blob = bucket.blob(f'raw-days/{gcs_file_name(date)}')

In [58]:
blob.upload_from_string(data.to_json())

In [56]:
data.to_json()

'{"1583193600000":24607.0,"1583197200000":23619.0,"1583200800000":23260.0,"1583204400000":23274.0,"1583208000000":23846.0,"1583211600000":26705.0,"1583215200000":30842.0,"1583218800000":33267.0,"1583222400000":34482.0,"1583226000000":34712.0,"1583229600000":34789.0,"1583233200000":34579.0,"1583236800000":34261.0,"1583240400000":33071.0,"1583244000000":32221.0,"1583247600000":31514.0,"1583251200000":31315.0,"1583254800000":31304.0,"1583258400000":33688.0,"1583262000000":34965.0,"1583265600000":33910.0,"1583269200000":31371.0,"1583272800000":28475.0}'

## Define the table

In [18]:
schema = [
    bigquery.SchemaField("timestamp", 'STRING', mode="REQUIRED"),
    bigquery.SchemaField("demand", "FLOAT")
]

table_id = 'ml-energy-dashboard.energy_loads.es_energy_demand'

table = bigquery.Table(table_id, schema=schema)
table = client_bq.create_table(table)

print(
    "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
)

Created table ml-energy-dashboard.energy_loads.es_energy_demand


## Initalize table with 2019 data

In [ ]:
dataaset_id = 'energy_loads'
#recreate the table with the passed csv
dataset_ref = client.dataset(dataset_id)
job_config = bigquery.LoadJobConfig()
job_config.autodetect = True
job_config.skip_leading_rows = 1

with open(local_path, 'rb') as source_file:
        job = client.load_table_from_file(
        source_file,
        table_ref,
        location='EU',  # Must match the destination dataset location.
        job_config=job_config)  # API request

job.result()  # Waits for table load to complete.

print('Loaded {} rows into project: {} dataset: {} table: {}.'.format(job.output_rows, project_id, dataset_id, table_name_bigquery))
